In [1]:
import datasets
import models

import numpy as np
import lasagne
import matplotlib.pyplot as plt
import seaborn as sns

import pickle

%matplotlib inline

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5110)


In [2]:
jfnet = models.JFnet()

In [3]:
layers = jfnet.net.values()

In [4]:
lasagne.layers.get_output_shape(layers)

[(None, 3, 512, 512),
 (None, 32, 256, 256),
 (None, 32, 256, 256),
 (None, 32, 127, 127),
 (None, 32, 127, 127),
 (None, 32, 127, 127),
 (None, 32, 127, 127),
 (None, 32, 127, 127),
 (None, 32, 63, 63),
 (None, 64, 63, 63),
 (None, 64, 63, 63),
 (None, 64, 63, 63),
 (None, 64, 63, 63),
 (None, 64, 31, 31),
 (None, 128, 31, 31),
 (None, 128, 31, 31),
 (None, 128, 31, 31),
 (None, 128, 31, 31),
 (None, 128, 31, 31),
 (None, 128, 31, 31),
 (None, 128, 31, 31),
 (None, 128, 31, 31),
 (None, 128, 15, 15),
 (None, 256, 15, 15),
 (None, 256, 15, 15),
 (None, 256, 15, 15),
 (None, 256, 15, 15),
 (None, 256, 15, 15),
 (None, 256, 15, 15),
 (None, 256, 15, 15),
 (None, 256, 15, 15),
 (None, 256, 7, 7),
 (None, 256, 7, 7),
 (None, 1024),
 (None, 512),
 (None, 2),
 (None, 514),
 (None, 1028),
 (None, 1028),
 (None, 1024),
 (None, 512),
 (None, 512),
 (None, 10),
 (None, 5),
 (None, 5)]

In [5]:
print("Total number of parameters: " + str(lasagne.layers.count_params(layers)))

Total number of parameters: 20923690


In [6]:
def get_params_shape(layers):
    param_values = lasagne.layers.get_all_param_values(layers)
    param_shapes = [params.shape for params in param_values]
    return param_shapes

In [7]:
# number conv layers and make custom names for dense layers
params_names = ["Conv_{}_{}".format(i // 2 + 1, n) for i, n in enumerate(lasagne.layers.get_all_params(layers)[:-6])]
params_names += ['Dense_1_W', 'Dense_1_b', 'Dense_2_W', 'Dense_2_b', 'Dense_3_W', 'Dense_3_b']
print(params_names)

['Conv_1_W', 'Conv_1_b', 'Conv_2_W', 'Conv_2_b', 'Conv_3_W', 'Conv_3_b', 'Conv_4_W', 'Conv_4_b', 'Conv_5_W', 'Conv_5_b', 'Conv_6_W', 'Conv_6_b', 'Conv_7_W', 'Conv_7_b', 'Conv_8_W', 'Conv_8_b', 'Conv_9_W', 'Conv_9_b', 'Conv_10_W', 'Conv_10_b', 'Conv_11_W', 'Conv_11_b', 'Conv_12_W', 'Conv_12_b', 'Conv_13_W', 'Conv_13_b', 'Dense_1_W', 'Dense_1_b', 'Dense_2_W', 'Dense_2_b', 'Dense_3_W', 'Dense_3_b']


In [8]:
# print total weight count for each trainable layers along with shape parameters take
shapes = [np.prod(param_shape) for param_shape in get_params_shape(layers)]
for l in zip(params_names, get_params_shape(layers), shapes):
    print(l)

('Conv_1_W', (32, 3, 7, 7), 4704)
('Conv_1_b', (32, 256, 256), 2097152)
('Conv_2_W', (32, 32, 3, 3), 9216)
('Conv_2_b', (32, 127, 127), 516128)
('Conv_3_W', (32, 32, 3, 3), 9216)
('Conv_3_b', (32, 127, 127), 516128)
('Conv_4_W', (64, 32, 3, 3), 18432)
('Conv_4_b', (64, 63, 63), 254016)
('Conv_5_W', (64, 64, 3, 3), 36864)
('Conv_5_b', (64, 63, 63), 254016)
('Conv_6_W', (128, 64, 3, 3), 73728)
('Conv_6_b', (128, 31, 31), 123008)
('Conv_7_W', (128, 128, 3, 3), 147456)
('Conv_7_b', (128, 31, 31), 123008)
('Conv_8_W', (128, 128, 3, 3), 147456)
('Conv_8_b', (128, 31, 31), 123008)
('Conv_9_W', (128, 128, 3, 3), 147456)
('Conv_9_b', (128, 31, 31), 123008)
('Conv_10_W', (256, 128, 3, 3), 294912)
('Conv_10_b', (256, 15, 15), 57600)
('Conv_11_W', (256, 256, 3, 3), 589824)
('Conv_11_b', (256, 15, 15), 57600)
('Conv_12_W', (256, 256, 3, 3), 589824)
('Conv_12_b', (256, 15, 15), 57600)
('Conv_13_W', (256, 256, 3, 3), 589824)
('Conv_13_b', (256, 15, 15), 57600)
('Dense_1_W', (12544, 1024), 12845056)
(

In [9]:
# save weights as a dictionary name-weights
weights = lasagne.layers.get_all_param_values(layers)
layer_weights_dict = dict(zip(params_names, weights))
layer_weights_dict['Dense_3_b']

array([ 0.59162462,  0.04647866,  0.40189323, -0.12301593, -0.41697878,
        0.63765049,  0.02411902,  0.39341599, -0.03656115, -0.51861304], dtype=float32)

In [10]:
weights= lasagne.layers.get_all_param_values(layers)

In [11]:
weights

[array([[[[ -6.16715401e-02,  -8.31918046e-02,  -1.04224913e-01, ...,
            -4.69745249e-02,   5.78085184e-02,   1.91566832e-02],
          [ -6.08775392e-02,  -1.22391187e-01,  -1.11354545e-01, ...,
             3.55320908e-02,  -3.32981721e-02,   4.94399481e-02],
          [ -1.18098982e-01,  -9.24304500e-02,  -7.81133249e-02, ...,
            -1.96847748e-02,   3.87467071e-02,   1.97796803e-02],
          ..., 
          [ -8.59301388e-02,   3.37873362e-02,   1.22304671e-01, ...,
             9.78811830e-02,  -4.42702398e-02,  -1.86790880e-02],
          [ -6.25792593e-02,   8.52711871e-03,   9.93065685e-02, ...,
             1.66835748e-02,  -1.61426347e-02,  -4.48718071e-02],
          [ -5.39366268e-02,  -5.47277145e-02,  -6.15704171e-02, ...,
            -1.78669989e-02,  -2.88721602e-02,  -2.97504831e-02]],
 
         [[ -4.04937603e-02,  -5.44766570e-03,  -2.30553038e-02, ...,
            -6.90462366e-02,  -5.90868443e-02,  -6.32727146e-02],
          [ -2.27178959e-03, 

In [12]:
np.save("JFnet_weights.npy", np.array(weights))